In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time

### Notes

#### Extra level of depth problem
<ul>
    <li> May have 4th level of depth, but this level may not be applicable everywhere </li>
    <li> Error catch to ensure this level is captured </li>
    <li> If this level not present for some instances then ensure the subsubsub level is None </li> 
</ul> 

#### What information is important in the dataframe
<ul>
    <li> Location categories - main upwards </li>
    <li> Time of depature</li>
</li>

### Main Code

In [2]:
usingWorkLaptop = False

In [3]:
#contains user_id,place_id and datetime of checkin 
if usingWorkLaptop:
    checkins = pd.read_csv("C:\\Users\\hcheena\\OneDrive - KPMG\\Documents\\UserMobilityMining\\gowalla\\gowalla_checkins.csv")
else:
    checkins = pd.read_csv("gowalla/gowalla_checkins.csv")

checkins.head()

,userid,placeid,datetime
0,1338,482954,2011-06-23T02:24:22Z
1,1338,580963,2011-06-22T14:23:03Z
2,1338,365256,2011-06-09T23:29:30Z
3,1338,89504,2011-05-22T15:54:30Z
4,1338,1267135,2011-05-21T16:51:13Z


In [4]:
#place_id, longitude, latitude
if usingWorkLaptop:
    placesSubset1 = pd.read_csv("C:\\Users\\hcheena\\OneDrive - KPMG\\Documents\\UserMobilityMining\\gowalla\\gowalla_spots_subset1.csv")
else:
    placesSubset1 = pd.read_csv("gowalla/gowalla_spots_subset1.csv")

placesSubset1.head()

,id,created_at,lng,lat,photos_count,checkins_count,users_count,radius_meters,highlights_count,items_count,max_items_count,spot_categories
0,8904,2008-12-06T16:28:53Z,-94.607499,39.052318,0,114,21,35,0,10,10,"[{'url': '/categories/89', 'name': 'Craftsman'}]"
1,8932,2008-12-13T02:16:51Z,-97.254356,32.927662,2,67,48,75,0,6,10,"[{'url': '/categories/17', 'name': 'BBQ'}]"
2,8936,2008-12-14T22:08:39Z,-94.591995,39.053318,0,75,46,75,0,10,10,"[{'url': '/categories/103', 'name': 'Theatre'}]"
3,8938,2008-12-15T00:22:49Z,-94.590311,39.052824,38,438,94,50,10,10,10,"[{'url': '/categories/1', 'name': 'Coffee Shop'}]"
4,8947,2008-12-16T23:14:05Z,-122.029631,37.331880,91,3100,1186,200,20,10,10,"[{'url': '/categories/121', 'name': 'Corporate..."


In [5]:
#load the category_structure to relate the places to the main category 

#load json file
if usingWorkLaptop:
    with open("C:\\Users\\hcheena\\OneDrive - KPMG\\Documents\\UserMobilityMining\\gowalla\\gowalla_category_structure.json") as f:
        categoryStructure = json.load(f)['spot_categories']
else:
    with open("gowalla/gowalla_category_structure.json") as f:
        categoryStructure = json.load(f)['spot_categories']

#create dataframe of category relationships
flattenJson = json_normalize(categoryStructure)

mainLevel = list()
secondLevel = list()
thirdLevel = list()
fourthLevel = list()

for firstIndex in range(0,flattenJson.shape[0]):
    #extract main/first level category
    mainCategory = flattenJson.loc[firstIndex]['name']
    #iterate through main categories 
    secondLevelInstance = flattenJson.loc[firstIndex]['spot_categories']
    
    if len(secondLevelInstance) == 0:
        mainLevel.append(mainCategory)
        secondLevel.append(None)
        thirdLevel.append(None)
    else:
        #iterate through second level categories
        for secondInstance in secondLevelInstance:
            #extract second level category
            secondLevelCategory = secondInstance['name']
            thirdLevelInstance = secondInstance['spot_categories']
            
            if len(thirdLevelInstance) == 0:
                mainLevel.append(mainCategory)
                secondLevel.append(secondLevelCategory)
                thirdLevel.append(None)
                
            else:
                #iterate through 3rd level categories
                for thirdInstance in thirdLevelInstance:
                    #extract third level category 
                    thirdLevelCategory = thirdInstance['name']
                    #add to lists
                    mainLevel.append(mainCategory)
                    secondLevel.append(secondLevelCategory)
                    thirdLevel.append(thirdLevelCategory)
            
#add to dataframe
categoryDf = pd.DataFrame({'Main':mainLevel, 'Second':secondLevel,'Third':thirdLevel})            

In [6]:
#extract the location information 
locationInformation = list()
for rowIndex in placesSubset1.index:
    
    rowInstance = placesSubset1['spot_categories'][rowIndex]
    firstIndex = rowInstance.rfind(":") + 3
    secondIndex = rowInstance.rfind("'")
    
    if len(locationInformation) == 0:
        locationInformation = [rowInstance[firstIndex:secondIndex]]
    else:
        locationInformation.append(rowInstance[firstIndex:secondIndex])
        
#dataframe created for checking purposes
locationDf = pd.DataFrame(locationInformation, columns=['Locations'])
locationDf.head()

,Locations
0,Craftsman
1,BBQ
2,Theatre
3,Coffee Shop
4,Corporate Office


### Process POI category information

In [ ]:
#assemble the dataframe with the location information and the rest of the location data
featuresToExtract = ['id','created_at', 'lng', 'lat']
locationFeatureDf = placesSubset1[featuresToExtract]

relatedMainCategory = list() 
relatedSecondCategory = list()
relatedThirdCategory = list()
indexUsed = list()

#add to locationDf the main, sub categories
#do do this locate whether the given location is in sub or subsub and then paste the associated main category value threre to create the entire dataframe

endTime = time.time()

#check search for location column wise 
for index in range(0,locationDf.shape[0]):
    
    if index % 10000 == 0:
        print(index)
        startTime = endTime #start time is equal to the previous end time
        endTime = time.time()
        print('Time Elapsed: ', endTime-startTime)
    
    searchLocation = locationDf['Locations'][index]
    
    #determine where in either mask the true value lies
    thirdMask = np.equal(searchLocation, categoryDf['Third'])
    secondMask = np.equal(searchLocation, categoryDf['Second'])
    mainMask = np.equal(searchLocation, categoryDf['Main'])
    searchMask = np.logical_or(np.logical_or(secondMask, thirdMask),mainMask)
    
    #if the location was not classified skip it and 
    if searchMask.any():
        #subset and then append to list
        mainCategory = categoryDf['Main'][searchMask].values[0] #extract value
        secondCategory = categoryDf['Second'][searchMask].values[0] #extract value
        thirdCategory = categoryDf['Third'][searchMask].values[0] #extract value

        relatedMainCategory.append(mainCategory)
        relatedSecondCategory.append(secondCategory)
        relatedThirdCategory.append(thirdCategory)
        indexUsed.append(index)
    else:
        #add missing location data 
        relatedMainCategory.append(np.NaN)
        relatedSecondCategory.append(np.NaN)
        relatedThirdCategory.append(np.NaN)
        indexUsed.append(np.NaN) 

relatedCategories = pd.DataFrame({'index':indexUsed,
                                  'mainCategory':relatedMainCategory,
                                  'secondLevelCategory':relatedSecondCategory,
                                 'thirdLevelCategory':relatedThirdCategory})

relatedCategories.head()

In [11]:
#assemble df

locationFeatureDf['relatedMainCategory'] = relatedMainCategory
locationFeatureDf['relatedSecondaryCategory'] = relatedSecondCategory
locationFeatureDf['relatedThirdCategory'] = relatedThirdCategory

/home/hasnain/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/hasnain/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/hasnain/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [12]:
locationFeatureDf.head()

,id,created_at,lng,lat,relatedMainCategory,relatedSecondaryCategory,relatedThirdCategory
0,8904,2008-12-06T16:28:53Z,-94.607499,39.052318,Community,Home,Craftsman
1,8932,2008-12-13T02:16:51Z,-97.254356,32.927662,Food,BBQ,None
2,8936,2008-12-14T22:08:39Z,-94.591995,39.053318,Entertainment,Theatre,None
3,8938,2008-12-15T00:22:49Z,-94.590311,39.052824,Food,Coffee Shop,None
4,8947,2008-12-16T23:14:05Z,-122.029631,37.331880,Community,Office,Corporate Office


In [13]:
#save locationFeatureDf
locationFeatureDf.to_csv("LocationFeatureData.csv")

### Checkin Date & Time Processing

In [ ]:
#create date-time feature
dateTime = checkins['datetime']
dateTime.head()

#split into date and time column 
#year format: YYYY-MM-DD 
#time format: HH:MM:SS

timeList = list()
dateList = list()

#convert year to fraction YYYY.fraction_of_year 
for index in range(0, dateTime.shape[0]):
    
    if index % 100000 == 0:
        print(index)
    
    #convert date to date object and then date fractional representation
    date = dateTime[index][:dateTime[index].find("T")] #extract date from datetime string 
    year = int(date[:date.find("-")])
    month = int(date[date.find("-")+1:date.rfind("-")])
    day = int(date[date.rfind("-")+1:])
    date = datetime.date(year, month, day) #create a date object
    #convert to fractional date
    numberOfDaysIntoTheYear = int(datetime.date.strftime(date,"%j"))
    if year % 4 == 0: #if leap year
        fractionalDate = year + numberOfDaysIntoTheYear / 366 #convert year to fractional representation
    else:
        fractionalDate = year + numberOfDaysIntoTheYear / 365 #convert year to fractional representation

    #convert time to time object and time fractional representation
    time = dateTime[index][dateTime[index].find("T")+1:-1] #extract time as string 
    hour = int(time[:time.find(":")])
    minute = int(time[time.find(":")+1:time.rfind(":")])
    second = int(time[time.rfind(":")+1:])
    time = datetime.time(hour, minute, second) #create a time object
    fractionalTime = int(datetime.time.strftime(time,"%H")) + int(datetime.time.strftime(time,"%M"))/60 #convert time to a fractional representation

    timeList.append(fractionalTime)
    dateList.append(fractionalDate)
    
checkins['Time'] = timeList
checkins['Date'] = dateList

In [5]:
checkins.head()

,userid,placeid,datetime,Time,Date
0,1338,482954,2011-06-23T02:24:22Z,2.400000,2011.476712
1,1338,580963,2011-06-22T14:23:03Z,14.383333,2011.473973
2,1338,365256,2011-06-09T23:29:30Z,23.483333,2011.438356
3,1338,89504,2011-05-22T15:54:30Z,15.900000,2011.389041
4,1338,1267135,2011-05-21T16:51:13Z,16.850000,2011.386301


In [7]:
checkins.to_csv("checkinData.csv")

### Connect location and users 

In [2]:
#import checkin data and location feature data
checkinDf = pd.read_csv("Data/checkinData.csv", index_col=0)

#drop columns
checkinDf = checkinDf.drop("datetime", axis=1)

checkinDf.head()

/home/hasnain/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,userid,placeid,Time,Date
0,1338,482954,2.400000,2011.476712
1,1338,580963,14.383333,2011.473973
2,1338,365256,23.483333,2011.438356
3,1338,89504,15.900000,2011.389041
4,1338,1267135,16.850000,2011.386301


In [3]:
locationFeatureDf = pd.read_csv("Data/LocationFeatureData.csv", index_col=0)

#drop columns 
locationFeatureDf = locationFeatureDf.drop("created_at", axis=1)
locationFeatureDf = locationFeatureDf.drop("relatedThirdCategory", axis=1)

locationFeatureDf.head()

,id,lng,lat,relatedMainCategory,relatedSecondaryCategory
0,8904,-94.607499,39.052318,Community,Home
1,8932,-97.254356,32.927662,Food,BBQ
2,8936,-94.591995,39.053318,Entertainment,Theatre
3,8938,-94.590311,39.052824,Food,Coffee Shop
4,8947,-122.029631,37.331880,Community,Office


In [4]:
#join locationFeatureDf and checkinDf at the place_id column
featureDf = checkinDf.join(locationFeatureDf.set_index('id'), on='placeid')
featureDf.head()

,userid,placeid,Time,Date,lng,lat,relatedMainCategory,relatedSecondaryCategory
0,1338,482954,2.400000,2011.476712,-93.177506,44.934198,Nightlife,Bar
1,1338,580963,14.383333,2011.473973,-93.366249,45.204928,Shopping,Gas & Automotive
2,1338,365256,23.483333,2011.438356,-93.277574,44.981997,Entertainment,Arena or Stadium
3,1338,89504,15.900000,2011.389041,-93.153680,44.982573,Entertainment,Zoo
4,1338,1267135,16.850000,2011.386301,-93.187120,44.948173,Shopping,Toys & Games


In [5]:
featureDf.to_csv("FeatureDf.csv")